Here we'll show a bit about how the convex hull algorithm works for 3D point sets.
As we'll see shortly, computing convex hulls is much more difficult in higher dimensions than in 2D because visibility gets more interesting.

In [ ]:
import numpy as np
from scipy.stats.qmc import PoissonDisk
rng = np.random.default_rng(seed=1729)
num_points = 40
poisson_disk = PoissonDisk(3, radius=0.05, seed=rng)
X = poisson_disk.random(num_points)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.scatter(*X.T);

In [ ]:
import zmsh
hull_machine = zmsh.convex_hull.ConvexHull(X)

In [ ]:
from copy import deepcopy
topologies = [deepcopy(hull_machine.topology)]
visible_cells_ids = []

while not hull_machine.is_done():
    vertex_id, cell_ids = hull_machine.visibility.get_next_point_and_cells()
    visible_cells_ids.append(cell_ids)
    hull_machine.step()
    topologies.append(deepcopy(hull_machine.topology))
    
visible_cells_ids.append([])

In [ ]:
topologies[-1]

Now we can visualize the progress of the algorithm.

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation
from matplotlib.colors import to_rgba
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

def animate(frame):
    cell_ids, topology = frame
    ax.clear()
    ax.view_init(azim=40)
    ax.scatter(*X.T, color="black")
    triangles_colors = [
        (s, "tab:orange" if index in cell_ids else "tab:blue")
        for index, s in enumerate(topology)
        if s.compressed().size != 0
    ]
    triangles, color_names = zip(*triangles_colors)
    colors = [to_rgba(c) for c in color_names]
    collection = Poly3DCollection(
        X[triangles, :], facecolors=colors, edgecolors="black", alpha=0.5
    )
    ax.add_collection(collection)

frames = zip(visible_cells_ids, topologies)
animation = FuncAnimation(fig, animate, frames, interval=1e3)

In [ ]:
from IPython.display import HTML
HTML(animation.to_jshtml())

Computing convex hulls in dimension 3 or higher is much more difficult in the 2D case because now multiple hull faces can be visible to the same extreme point.